<a href="https://colab.research.google.com/github/Maxxx-VS/Test_task/blob/master/Sushi_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тестовое задание

Постановка задачи:
В оффлайн-продажах важно оценивать предполагаемый поток клиентов и объем проданной продукции. В рамках данного тестового задания необходимо выполнить следующие шаги:
1.	На основании предоставленных исходных данных выберите и сформируйте список уникальных номенклатурных позиций, которые присутствуют в продажах.
2.	Подготовьте «силуэт продаж» – выявите и опишите характерный профиль продаж по каждой номенклатурной позиции, учитывая сезонность по дням недели и месяцам. Силуэт должен отражать типичные изменения объема продаж во времени.
3.	Используя сформированный силуэт продаж, подготовьте предварительный прогноз объема продаж на завтрашний день для каждой номенклатурной позиции (завтрашним считается день, следующий за днем сдачи тестового задания).
4.	С помощью инструментов машинного обучения проведите пост-обработку и корректировку предварительного прогноза, полученного по силуэту продаж, до более точного значения.
Предоставьте итоговый результат в виде таблицы с перечнем уникальных номенклатурных позиций и соответствующим прогнозом продаж. Дополнительно кратко опишите используемый подход и методику расчета и корректировки прогноза.
Из 3 представленных на выбор файлов (данные за прошлый год по трем разным торговым точкам), выбрать нужно только один (в решении можно использовать все три).

Описание переменных:
CloseTime – время чека
DishAmountInt – количество позиций (целевая переменная)
DishDiscountSumInt – сумма в рублях
DishName – название блюда
OpenDate.Typed– дата заказа
RestorauntGroup– название торговой точки

Рекомендации:
Цель данного тестового посмотреть на ход решения и рассуждений.
Укажите, какие переменные используются для построения силуэта продаж и модели, почему именно они, какие подходы используются в решении (приветствуется добавление собственных признаков). Никаких ограничений в плане выбора технологий прогнозирования нет (кроме, конечно, здравого смысла). Как итог нужно презентовать код решения с пояснениями (возможно, ноутбук в google colab или jupyter) и ответить на возникающие вопросы. Готовить слайды рр на данном этане не нужно.


In [ ]:
'''ЗАГРУЗКА ДАТАСЕТА.
   Мной выбрана загрузка табличных данных, сохраненных на Google Drive.
   Из трех файлов - выбрал самый большой'''

